In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
import gymnasium as gym

from astropy.io import fits
from datetime import datetime, timezone
import pandas as pd
import json
import fitsio
import time
import pickle
import re

%reload_ext autoreload
%autoreload 2

In [4]:
import survey_ops
from survey_ops.utils import units, geometry, interpolate
from survey_ops.coreRL.offline_dataset import OfflineDECamDataset
from survey_ops.coreRL.agents import Agent
from survey_ops.algorithms import DDQN, BehaviorCloning
from survey_ops.utils.sys_utils import seed_everything
from survey_ops.coreRL.data_processing import load_raw_data_to_dataframe


In [5]:
from survey_ops.utils import ephemerides
from tqdm import tqdm
from pathlib import Path

In [6]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from collections import Counter
from scipy.stats import entropy

In [7]:
SEED = 10
seed_everything(SEED)
torch.set_default_dtype(torch.float32)
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"   
)

# Load config and lookup files

In [8]:
results_outdir = '../experiment_results/unit_tests/bc-azel-test/'

In [9]:
with open('../configs/global_config.json', 'r') as f:
    gcfg = json.load(f)

In [10]:
with open(results_outdir + 'config.json', 'r') as f:
    cfg = json.load(f)

In [11]:
nside = cfg['data']['nside']

In [12]:
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2NVISITS'], 'r') as f:
#     field2nvisits = json.load(f)
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2NAME'], 'r') as f:
#     field2name = json.load(f)
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2RADEC'], 'r') as f:
#     field2radec = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2azel.json', 'r') as f:
    bin2azel = json.load(f)
with open(f'../data/lookups/nside{nside}_bin2radec.json', 'r') as f:
    bin2radec = json.load(f)


In [13]:
# fits_path = Path(cfg.get('paths.DFITS')).resolve().parents[1] / 'data' / cfg.get('paths.DFITS')
# json_path = Path(cfg.get('paths.DJSON')).resolve().parents[1] / 'data' / cfg.get('paths.DJSON')

df = load_raw_data_to_dataframe(fits_path=None, json_path='../data/fits/decam-exposures-20251211.json')

d = OfflineDECamDataset(
    df=df,
    cfg=cfg,
    gcfg=gcfg,
)

Calculating sun and moon ra/dec and az/el: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5017/5017 [00:04<00:00, 1197.46it/s]
Calculating bin features for all healpix bins and timestamps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5017/5017 [00:00<00:00, 8602953.05it/s]
Normalizing bin features: 0it [00:00, ?it/s]


In [13]:
d.visit_history

AttributeError: 'OfflineDECamDataset' object has no attribute 'visit_history'

In [21]:
d._df

,expnum,ra,dec,exptime,filter,propid,program,object,teff,fwhm,...,sun_ra_cos,sun_ra_sin,sun_az_cos,sun_az_sin,moon_ra_cos,moon_ra_sin,moon_az_cos,moon_az_sin,lst_cos,lst_sin
0,NaN,5.892551,-0.528213,NaN,NaN,NaN,NaN,zenith,NaN,NaN,...,-0.753933,-0.656951,-0.466793,-0.884367,-0.925526,-0.378684,-0.508581,-0.861014,0.590954,0.806705
1,791159.0,5.899418,-0.000193,90.0,r,2012B-0001,survey,DES survey hex -219-4 tiling 10,0.25,1.34,...,-0.753932,-0.656953,-0.467179,-0.884163,-0.925513,-0.378715,-0.509078,-0.860721,0.590861,0.806774
2,791160.0,5.899417,-0.000191,90.0,g,2012B-0001,survey,DES survey hex -219-4 tiling 10,0.22,1.25,...,-0.753916,-0.656971,-0.471781,-0.881716,-0.925363,-0.379081,-0.515006,-0.857187,0.589746,0.807589
3,791161.0,6.066687,-0.000193,90.0,r,2012B-0001,survey,DES survey hex -124-4 tiling 10,0.42,1.15,...,-0.753897,-0.656993,-0.477372,-0.878702,-0.925181,-0.379525,-0.522219,-0.852811,0.588395,0.808573
4,791162.0,6.066684,-0.000192,90.0,g,2012B-0001,survey,DES survey hex -124-4 tiling 10,0.28,1.36,...,-0.753881,-0.657011,-0.482010,-0.876166,-0.925031,-0.379893,-0.528215,-0.849111,0.587278,0.809385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5012,807659.0,1.039591,-1.150495,90.0,i,2012B-0001,survey,DES survey hex 577-663 tiling 9,0.68,0.98,...,0.178136,-0.984006,-0.997606,-0.069161,-0.846504,-0.532382,-0.330881,0.943672,0.963859,0.266412
5013,807660.0,1.047130,-1.150789,90.0,i,2012B-0001,survey,DES survey hex 577-663 tiling 7,0.68,0.97,...,0.178162,-0.984001,-0.998241,-0.059279,-0.846318,-0.532678,-0.326312,0.945262,0.963490,0.267743
5014,807661.0,1.145475,-1.151089,90.0,i,2012B-0001,survey,DES survey hex 657-663 tiling 10,0.72,0.93,...,0.178190,-0.983996,-0.998809,-0.048800,-0.846121,-0.532990,-0.321494,0.946912,0.963098,0.269151
5015,807662.0,1.145468,-1.151081,90.0,g,2012B-0001,survey,DES survey hex 657-663 tiling 10,0.96,1.10,...,0.178217,-0.983991,-0.999247,-0.038807,-0.845934,-0.533287,-0.316924,0.948451,0.962722,0.270492
